In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [11]:
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [13]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [14]:
# Convolutional Neural Network 모델을 정의합니다.
class UserCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 흑백 이미지라 in_channels=1, 채널 수(=커널 수)는 out_channels=32개, 커널 크기가 3이므로 패딩은 1, 보폭 stride는 디폴트 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # 앞서 풀링층이 있을테지만 conv1에서 오는 피쳐맵 크기에는 변화를 주지 않음.
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # Flatten을 거치면, 채널/커널 수(64) * 이미지 행크기(7) * 이미지열크기(7)
        self.linear1 = nn.Linear(in_features=64*8*8, out_features=512)
        self.linear2 = nn.Linear(in_features=512, out_features=128)
        self.linear3 = nn.Linear(in_features=128, out_features=10)
        self.relu = nn.ReLU()
        # 보폭 stride가 2이므로 풀링을 거치면 이미지 크기는 절반으로 줄어들게 됨
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.softmax = nn.LogSoftmax()
        # 직렬화를 위해 사용
        self.flatten = nn.Flatten()

    def forward(self, x):
        
        #Building Block 1
        x = self.conv1(x) # in_img_size=(32,32), in_channels=3, 
                          # out_channels=32, kernel_size=3, padding=1, out_img_size=(32,32)
        x = self.relu(x)  # in_img_size=(32,32), out_channels=32, out_img_size=(32,32)
        x = self.pool(x) # in_img_size=(32,32), in_channels=32, kernel_size=2, stride=2
                          # out_channels=32,out_img_size=(16,16)
        
        #Building Block 2 
        x = self.conv2(x) # in_img=(16,16), in_channels=32, out_channels=64, kernel_size=3, stride=1
                          # out_img_size=(16,16), out_channels=64
        x = self.relu(x) # out_img_size=(16,16), out_channels=64
        x = self.pool(x) # in_img_size=(16,16), out_channels=64, kernel_size=2, stride=2
                          # out_img_size=(8,8), out_channels=64
                          
        x = self.conv3(x) # in_img=(8,8), in_channels=64, out_channels=64, kernel_size=3, stride=1
                          # out_img_size=(8,8), out_channels=64
        x = self.relu(x) # out_img_size=(8,8), out_channels=64
                           
        #Serialization for 2D image * channels                           
        x = self.flatten(x) # in_img_size=(8,8), in_channels=64
                            # out_img_size=(64*8*8)
                            
        #Fully connected layers
        x = self.linear1(x) #in_features=3136, out_features=512
        x = self.relu(x) #in_features=512, out_features=512
        
        x = self.linear2(x) #in_features=512, out_features=128
        x = self.relu(x) #in_features=128, out_features=128
        
        #out layer
        x = self.linear3(x) #in_features=128, out_features=10
        
        # 분류문제에서는 CrossEntropy(Lossfunc) + softmax 출력 활성화 함수 조합
        x = self.softmax(x) #in_features=10, out_features=10
        return x

model = UserCNN().to(device)
print(model)

UserCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear1): Linear(in_features=4096, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (softmax): LogSoftmax(dim=None)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [15]:
model = UserCNN().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

UserCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear1): Linear(in_features=4096, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (softmax): LogSoftmax(dim=None)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # inference 모드로 실행하기 위해 학습시에 필요한 Dropout, batchnorm등의 기능을 비활성화함
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): # autograd engine(gradient를 계산해주는 context)을 비활성화함
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------


/home/gjlee/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 2.305349  [   64/50000]
loss: 1.810338  [ 6464/50000]
loss: 1.642668  [12864/50000]
loss: 1.612566  [19264/50000]
loss: 1.451246  [25664/50000]
loss: 1.463691  [32064/50000]
loss: 1.274773  [38464/50000]
loss: 1.379895  [44864/50000]
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.190510 

Epoch 2
-------------------------------
loss: 1.142143  [   64/50000]
loss: 1.105086  [ 6464/50000]
loss: 0.924566  [12864/50000]
loss: 1.064249  [19264/50000]
loss: 0.947034  [25664/50000]
loss: 1.204810  [32064/50000]
loss: 0.935847  [38464/50000]
loss: 1.127786  [44864/50000]
Test Error: 
 Accuracy: 64.3%, Avg loss: 0.989626 

Epoch 3
-------------------------------
loss: 0.855717  [   64/50000]
loss: 0.892665  [ 6464/50000]
loss: 0.770022  [12864/50000]
loss: 0.936392  [19264/50000]
loss: 0.811818  [25664/50000]
loss: 0.991279  [32064/50000]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")